<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# DKN : Deep Knowledge-Aware Network for News Recommendation
DKN \[1\] is a deep learning model which incorporates information from knowledge graph for better news recommendation. Specifically, DKN uses TransX \[2\] method for knowledge graph representaion learning, then applies a CNN framework, named KCNN, to combine entity embedding with word embedding and generate a final embedding vector for a news article. CTR prediction is made via an attention-based neural scorer. 

## Properties of DKN:
- DKN is a content-based deep model for CTR prediction rather than traditional ID-based collaborative filtering. 
- It makes use of knowledge entities and common sense in news content via joint learning from semantic-level and knnowledge-level representations of news articles.
- DKN uses an attention module to dynamically calculate a user's aggregated historical representaition.

## Data format:
One simple example: <br>
`0 CandidateNews:1050,5238,5165,6996,1922,7058,7263,0,0,0 entity:0,0,0,0,2562,2562,2562,0,0,0 clickedNews0:7680,3978,1451,409,4681,0,0,0,0,0 entity0:0,0,0,395,0,0,0,0,0,0 clickedNews1:3698,2301,1055,6481,6242,7203,0,0,0,0 entity1:0,0,1904,1904,1904,0,0,0,0,0 ...`
<br>
In general, each line in data file represents one instance, the format is like: <br> 
`[label] [CandidateNews:w1,w2,w3,...] [entity:e1,e2,e3,...] [clickedNews0:w1,w2,w3,...] [entity0:e1,e2,e3,...] ...` <br>
It contains several parts seperated by space, i.e. label part, CandidateNews part, ClickedHistory part.  CandidateNews part describes the target news article we are going to score in this instance, it is represented by (aligned) title words and title entities. To take a quick example, a news title may be : `Trump to deliver State of the Union address next week` , then the title words value may be `CandidateNews:34,45,334,23,12,987,3456,111,456,432` and the title entitie value may be: `entity:45,0,0,0,0,0,0,0,0,0`. Only the first value of entity vector is non-zero due to the word `Trump`. <br>
clickedNewsk and entityk describe the k-th news article the user ever clicked and the format is the same as candidate news. Words and entities are aligned in news title. We use a fixed length to describe an article, if the title is less than the fixed length, just pad it with zeros.

## Global settings and imports

In [2]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import *
from reco_utils.recommender.deeprec.models.dkn import *
from reco_utils.recommender.deeprec.io.dkn_iterator import *
import papermill as pm


## Download and load data

In [4]:
data_path = '../../tests/resources/deeprec/dkn'
yaml_file = os.path.join(data_path, r'dkn.yaml')
train_file = os.path.join(data_path, r'final_train_with_entity.txt')
valid_file = os.path.join(data_path, r'final_test_with_entity.txt')
wordEmb_file = os.path.join(data_path, r'word_embeddings_100.npy')
entityEmb_file = os.path.join(data_path, r'TransE_entity2vec_100.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/deeprec/', data_path, 'dknresources.zip')
    
    

## Create hyper-parameters

In [5]:
epoch=10

In [6]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, entityEmb_file=entityEmb_file, epochs=epoch)
print(hparams)

[('DNN_FIELD_NUM', None), ('FEATURE_COUNT', None), ('FIELD_COUNT', None), ('MODEL_DIR', None), ('PAIR_NUM', None), ('SUMMARIES_DIR', None), ('activation', ['sigmoid']), ('attention_activation', 'relu'), ('attention_dropout', 0.0), ('attention_layer_sizes', 100), ('batch_size', 100), ('cross_activation', 'identity'), ('cross_l1', 0.0), ('cross_l2', 0.0), ('cross_layer_sizes', None), ('cross_layers', None), ('data_format', 'dkn'), ('dim', 100), ('doc_size', 10), ('dropout', [0.0]), ('dtype', 32), ('embed_l1', 0.0), ('embed_l2', 0.0005), ('enable_BN', False), ('entityEmb_file', '../../tests/resources/deeprec/dkn\\TransE_entity2vec_100.npy'), ('entity_dim', 100), ('entity_embedding_method', 'TransE'), ('entity_size', 3777), ('epochs', 10), ('fast_CIN_d', 0), ('filter_sizes', [1, 2, 3]), ('init_method', 'uniform'), ('init_value', 0.1), ('is_clip_norm', 0), ('iterator_type', None), ('kg_file', None), ('kg_training_interval', 5), ('layer_l1', 0.0), ('layer_l2', 0.0005), ('layer_sizes', [300])

In [6]:
input_creator = DKNTextIterator

## Train the DKN model

In [15]:
model = DKN(hparams, input_creator)

In [11]:
print(model.run_eval(valid_file))

{'auc': 0.5029, 'acc': 0.4275, 'f1': 0.0, 'logloss': 0.7688}


C:\ProgramData\Anaconda3\envs\reco\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
model.fit(train_file, valid_file)

at epoch 1 train info: acc:0.6061, auc:0.7798, f1:0.7548, logloss:0.6705 eval info: acc:0.5725, auc:0.5484, f1:0.7281, logloss:0.6847
at epoch 1 , train time: 98.8 eval time: 37.0
at epoch 2 train info: acc:0.6061, auc:0.7643, f1:0.7548, logloss:0.6705 eval info: acc:0.5725, auc:0.5607, f1:0.7281, logloss:0.6846
at epoch 2 , train time: 82.1 eval time: 36.7
at epoch 3 train info: acc:0.6061, auc:0.7453, f1:0.7548, logloss:0.6704 eval info: acc:0.5725, auc:0.5651, f1:0.7281, logloss:0.6846
at epoch 3 , train time: 81.9 eval time: 36.4
at epoch 4 train info: acc:0.6061, auc:0.7339, f1:0.7548, logloss:0.6704 eval info: acc:0.5725, auc:0.5733, f1:0.7281, logloss:0.6846
at epoch 4 , train time: 82.4 eval time: 35.1
at epoch 5 train info: acc:0.6061, auc:0.7287, f1:0.7548, logloss:0.6704 eval info: acc:0.5725, auc:0.5839, f1:0.7281, logloss:0.6846
at epoch 5 , train time: 81.4 eval time: 34.9
at epoch 6 train info: acc:0.6061, auc:0.7278, f1:0.7548, logloss:0.6704 eval info: acc:0.5725, auc:

Now we can test again the performance on valid set:

In [17]:
res = model.run_eval(valid_file)
print(res)
pm.record("res", res)

{'auc': 0.5899, 'acc': 0.5725, 'f1': 0.7281, 'logloss': 0.6845}


## Reference
\[1\] Wang, Hongwei, et al. "DKN: Deep Knowledge-Aware Network for News Recommendation." Proceedings of the 2018 World Wide Web Conference on World Wide Web. International World Wide Web Conferences Steering Committee, 2018.<br>
\[2\] Knowledge Graph Embeddings including TransE, TransH, TransR and PTransE. https://github.com/thunlp/KB2E